In [46]:
import openpyxl
import openpyxl.styles
import openpyxl.utils

import openpyxl.worksheet.formula
import re

In [47]:
file_name = "bradford.xlsx"
distribution_name = file_name.split(".")[0]

In [48]:
workbook_template = openpyxl.load_workbook(f"../excel-files/{file_name}")
sheet_0_template = workbook_template[workbook_template.sheetnames[0]]
sheet_1_template = workbook_template[workbook_template.sheetnames[1]]
sheet_2_template = workbook_template[workbook_template.sheetnames[2]]

In [49]:
def change_formula(formula: str | None):
    if formula is None:
        return None
    cells = {
        "$C$3": "$D$6",
        "$C$4": "$D$7",
        "$C$5": "$D$8",
        "$C$6": "$D$9",
        "C3": "$D$6",
        "C4": "$D$7",
        "C5": "$D$8",
        "C6": "$D$9",
        "_xlfn.ANCHORARRAY(E3)": "G6",
        "E3": "G6",
        "M4": "$C$30",
        "M5": "$C$31",
        "M6": "$C$32",
        "M7": "$C$33",
        "L4": "B30",
        "L5": "B31",
        "L6": "B32",
        "L7": "B33",
        "J4": "$D$14",
    }
    r = formula
    for cell in cells.keys():
        r = r.replace(cell, cells[cell])
    return r

In [50]:
ppf_formula = sheet_0_template.cell(row=3, column=6).value
pdf_formula = sheet_1_template.cell(row=3, column=6).value.text
cdf_formula = sheet_2_template.cell(row=3, column=6).value.text

mean_formula = sheet_0_template.cell(row=3, column=10).value
variance_formula = sheet_0_template.cell(row=4, column=10).value
standard_deviation_formula = sheet_0_template.cell(row=5, column=10).value
skewness_formula = sheet_0_template.cell(row=6, column=10).value
kurtosis_formula = sheet_0_template.cell(row=7, column=10).value
median_formula = sheet_0_template.cell(row=8, column=10).value
mode_formula = sheet_0_template.cell(row=9, column=10).value

nc_miu1_formula = sheet_0_template.cell(row=4, column=13).value
nc_miu2_formula = sheet_0_template.cell(row=5, column=13).value
nc_miu3_formula = sheet_0_template.cell(row=6, column=13).value
nc_miu4_formula = sheet_0_template.cell(row=7, column=13).value


In [51]:
if nc_miu1_formula is None:
    workbook_file = openpyxl.load_workbook("./template_no_moments.xlsx")
else:
    workbook_file = openpyxl.load_workbook("./template_moments.xlsx")

sheet_0_file = workbook_file[workbook_file.sheetnames[0]]
sheet_1_file = workbook_file[workbook_file.sheetnames[1]]
sheet_2_file = workbook_file[workbook_file.sheetnames[2]]
sheet_3_file = workbook_file[workbook_file.sheetnames[3]]

In [52]:
def obtener_maximo_valor_columna(workbook, num_hoja, letra_columna):
    sheet = workbook[workbook.sheetnames[num_hoja]]
    return max((c.row for c in sheet[letra_columna] if c.value is not None))

In [53]:
n_params = obtener_maximo_valor_columna(workbook_template, 0, "B") - 2
for row in range(3, 3 + n_params):
    sheet_0_file.cell(row=row + 3, column=2).value = sheet_0_template.cell(row=row, column=2).value
    sheet_0_file.cell(row=row + 3, column=3).value = "R+"
    sheet_0_file.cell(row=row + 3, column=4).value = sheet_0_template.cell(row=row, column=3).value

    # sheet_0_file.cell(row=row + 3, column=2).font = openpyxl.styles.Font(name="Roboto", size=10)
    # sheet_0_file.cell(row=row + 3, column=2).alignment = openpyxl.styles.Alignment(horizontal="center")
    # sheet_0_file.cell(row=row + 3, column=3).font = openpyxl.styles.Font(name="Roboto", size=10, italic=True)
    # sheet_0_file.cell(row=row + 3, column=3).alignment = openpyxl.styles.Alignment(horizontal="center")
    # sheet_0_file.cell(row=row + 3, column=4).font = openpyxl.styles.Font(name="Roboto", size=10)
    # sheet_0_file.cell(row=row + 3, column=4).alignment = openpyxl.styles.Alignment(horizontal="center")


In [54]:
for row in range(3, 3 + n_params):
    for sheet_name in workbook_file.sheetnames[1:]:
        sheet = workbook_file[sheet_name]
        sheet.cell(row=row + 3, column=2).value = f"='Random Generation'!B{row+3}"
        sheet.cell(row=row + 3, column=3).value = f"='Random Generation'!C{row+3}"
        sheet.cell(row=row + 3, column=4).value = f"='Random Generation'!D{row+3}"

        # sheet.cell(row=row + 3, column=2).font = openpyxl.styles.Font(name="Roboto", size=10)
        # sheet.cell(row=row + 3, column=2).alignment = openpyxl.styles.Alignment(horizontal="center")
        # sheet.cell(row=row + 3, column=3).font = openpyxl.styles.Font(name="Roboto", size=10, italic=True)
        # sheet.cell(row=row + 3, column=3).alignment = openpyxl.styles.Alignment(horizontal="center")
        # sheet.cell(row=row + 3, column=4).font = openpyxl.styles.Font(name="Roboto", size=10)
        # sheet.cell(row=row + 3, column=4).alignment = openpyxl.styles.Alignment(horizontal="center")

In [55]:
def integration_formula_replace(formula: str):
    replacements = {
        "_xlfn.NORM.S.DIST": "DISTR.NORM.ESTAND.N",
        "_xlfn.BETA.DIST": "DISTR.BETA.N",
        "_xlfn.GAMMA": "GAMMA",
        "SQRT": "RAIZ",
        "_xlfn.LAMBDA": "LAMBDA",
        "_xlfn.MAP": "MAP",
        "_xlfn.T.DIST": "DISTR.T.N",
        "TRUE": "VERDADERO",
        "FALSE": "FALSO",
        ",": ";",
        "=": "",
        "_xlfn.ANCHORARRAY(G6)": "x",
        "G6": "x",
        "_xlpm.x": "x",
        "0.5": "0,5",
        "1.5": "1,5",
        "IF": "SI",
        "GAMMALN": "GAMMA.LN"
    }
    regex = re.compile("(%s)" % "|".join(map(re.escape, replacements.keys())))
    return regex.sub(lambda mo: replacements[mo.string[mo.start():mo.end()]], formula) 

In [56]:
parameters = [sheet_0_template.cell(row=row, column=2).value for row in range(3, 3 + n_params)]

In [57]:
def snake_to_pascal(snake_str):
    components = snake_str.split("_")
    pascal_str = "".join(x.title() for x in components)
    return pascal_str


In [58]:
DISTRIBUTION_NOTATION = f"X~{snake_to_pascal(distribution_name)}({','.join(parameters)})"

In [59]:
sheet_0_file["H6"].value = change_formula(ppf_formula)
sheet_1_file["H6"].value = change_formula(pdf_formula)
sheet_2_file["H6"].value = change_formula(cdf_formula)
sheet_3_file["I6"].value = change_formula(ppf_formula).replace("G6", "H6")
sheet_0_file["D13"].value = change_formula(mean_formula)
sheet_0_file["D14"].value = change_formula(variance_formula)
sheet_0_file["D15"].value = change_formula(standard_deviation_formula)
sheet_0_file["D16"].value = change_formula(skewness_formula)
sheet_0_file["D17"].value = change_formula(kurtosis_formula)
sheet_0_file["D18"].value = change_formula(median_formula)
sheet_0_file["D19"].value = change_formula(mode_formula)

if nc_miu1_formula is None:
    sheet_0_file["C30"].value = f"INTEGRATION_QUAD(LAMBDA(x;(x^B30)*({integration_formula_replace(change_formula(pdf_formula))}));MIN(H6:H10005);MAX(H6:H10005))"
    sheet_0_file["C31"].value = f"INTEGRATION_QUAD(LAMBDA(x;(x^B31)*({integration_formula_replace(change_formula(pdf_formula))}));MIN(H6:H10005);MAX(H6:H10005))"
    sheet_0_file["C32"].value = f"INTEGRATION_QUAD(LAMBDA(x;(x^B32)*({integration_formula_replace(change_formula(pdf_formula))}));MIN(H6:H10005);MAX(H6:H10005))"
    sheet_0_file["C33"].value = f"INTEGRATION_QUAD(LAMBDA(x;(x^B33)*({integration_formula_replace(change_formula(pdf_formula))}));MIN(H6:H10005);MAX(H6:H10005))"
else:
    sheet_0_file["C30"].value = change_formula(nc_miu1_formula)
    sheet_0_file["C31"].value = change_formula(nc_miu2_formula)
    sheet_0_file["C32"].value = change_formula(nc_miu3_formula)
    sheet_0_file["C33"].value = change_formula(nc_miu4_formula)

sheet_0_file["H5"].value = DISTRIBUTION_NOTATION
sheet_1_file["G5"].value = DISTRIBUTION_NOTATION
sheet_2_file["G5"].value = DISTRIBUTION_NOTATION
sheet_3_file["G5"].value = DISTRIBUTION_NOTATION

sheet_0_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper().replace("_", " "))
sheet_1_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper().replace("_", " "))
sheet_2_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper().replace("_", " "))
sheet_3_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper().replace("_", " "))

# sheet_2_file["I6"].value = "=F6/COUNT($F$6:$F$10005)"
# sheet_3_file["H6"].value = "=(F6-1/2)/COUNT($F$6:$F$10005)"

sheet_1_file["G6"].value = "'=SORT('Random Generation'!H6:H10005)"
sheet_2_file["G6"].value = "'=SORT('Random Generation'!H6:H10005)"
sheet_3_file["G6"].value = "'=SORT('Random Generation'!H6:H10005)"

In [60]:
workbook_file.save(f"./{file_name}")